In [1]:
using IntervalArithmetic
using Plots 
using Dates
using StatsBase

## Common functions

In [2]:
function phi_n(phi,T,x, s, n)
    "Calculate the image of phi^{(n)}"
    if n == 0
        return s
    else
        return phi.image(x, phi_n(phi,T,T.image(x), s, n-1))
    end
end


phi_n (generic function with 1 method)

In [3]:
function phi_nBoundK(phi,T,x, s, n,K)
    if n == 0
        return s
    else
        return phi.image(x,min(phi_nBoundK(phi,T,T.image(x), s, n-1,K),K))
    end
end

phi_nBoundK (generic function with 1 method)

In [4]:
function phi_n_per(phi,orb,len, s, n,i,err)
    "Calculate the image of phi^{(n)} for a periodic orbit"
    if n == 0
        return interval(s,s)
    else
        return phi.image(interval(orb[i+1]-err,orb[i+1]+err), phi_n_per(phi,orb,len, s, n-1,(i+1)%len,err))
    end
end


phi_n_per (generic function with 1 method)

In [5]:
function compose(f, x, n)
    if n == 0
        return x
    else
        return compose(f, f(x), n-1)
    end
end


compose (generic function with 1 method)

## Upper bound of q

In [6]:
function FindK(phi,T,N)
    K=1/2
    A=[interval((i-1)/2^N,i/2^N) for i in 1:2^N]
    while K<0.9999
        r=Inf
        for NK in 1:12
            ph=maximum([sup(phi_n(phi,T,x, K, NK)) for x in A])
            if K>ph
                r=min(r,ph)
            end
        end
        if r!=Inf
            return r
        end
        K=(1+K)/2
    end
    error("K not find")
end

FindK (generic function with 1 method)

In [7]:
function upboundq(phi,T,n,N,plt)  # N>n
    "find a good upper bound of q"
    K=FindK(phi,T,N)
    l1=[[interval(i/2^N,(i+1)/2^N),interval(K,K)]  for i in 0:(2^N-1)]
    l2=[phi_n(phi,T,x[1],x[2],n) for x in l1]
    if plt==true
         x=[i/2^N for i in 1:2^N]
        majqm=[sup(i) for i in l2]
        p=plot(x,majqm,title="upboundq",ylim=(-0.01,1.01))
        display(p)
    end
    return l2
end 


upboundq (generic function with 1 method)

## Upper bound of LambdaF

In [8]:
function upboundlambdaF(phi,T,n,N,m,Q,I)
    Indice=[i for i in 1:2^N]
    Nmaj=N
    bound=-Inf
    K=Inf
    for z in 1:I
        if Nmaj<=16
            K=FindK(phi,T,Nmaj)
        end
        len=length(Indice)
        l=[interval((i-1)/2^Nmaj,i/2^Nmaj) for i in Indice]
        Tl=[]
        r=[interval(0,0) for i in 1:len]
        infi=[0 for i in 1:len]
        for i in 1:m
            Tl=[T.image(x) for x in l]
            majq=[phi_nBoundK(phi,T,x,interval(K,K),n,K) for x in Tl]
            r+=[phi.logds(l[j],majq[j]) for j in 1:len]
            infi=[min(sup(r[j])/i,infi[j]) for j in 1:len]
            l=Tl
        end
        boundmaj=quantile(infi,1-1/2^Q)
        if bound>boundmaj || boundmaj==0
            error("bound")
        end
        bound=boundmaj
        if z==I
            return maximum(infi)
        end
        Nmaj=Nmaj+Q
        Indicemaj=Indice[findall(x -> x > bound, infi)].*2^Q
        Indice=reduce(vcat,[Indicemaj.-i for i in 0:2^Q-1])
    end
end

upboundlambdaF (generic function with 1 method)

## Lower bound of q 

In [9]:
function lowboundq(phi,T,n,N,plt)  # N>n
    "find a good lower bound of q"
    l1=[[interval(i/2^N,(i+1)/2^N),interval(0,0)]  for i in 0:(2^N-1)]
    l2=[phi_n(phi,T,x[1],x[2],n) for x in l1]
    if plt==true
        x=[i/2^N for i in 1:2^N]
        minqm=[inf(i) for i in l2]
        p=plot(x,minqm,title="lowboundq",ylim=(-0.01,1.01))
        display(p)
    end
    return l2
end

lowboundq (generic function with 1 method)

## Bounds on q 

In [10]:
function boundq(phi,T,n,N,plt)  # N>n
    "find good bounds of q"
        K=FindK(phi,T,N)
        l1=[[interval(i/2^N,(i+1)/2^N),interval(K,K)]  for i in 0:(2^N-1)]
        l2=[phi_n(phi,T,x[1],x[2],n) for x in l1]
        l3=[[interval(i/2^N,(i+1)/2^N),interval(0,0)]  for i in 0:(2^N-1)]
        l4=[phi_n(phi,T,x[1],x[2],n) for x in l3]
        if plt==true
            x=[i/2^N for i in 1:2^N]
            majqm=[sup(i) for i in l2]
            minqm=[inf(i) for i in l4]
            plot(x,majqm,label="upboundq",ylim=(-0.01,1.01))
            display(plot!(x,minqm,label="lowboundq"))
        end
        return l2,l4
    end

boundq (generic function with 1 method)

## Lower bound of Lambda_F

In [11]:
"finds a representative of each sequence of size n with integer values 
between 0 and d up to rotation and not containing a repeating pattern"

function is_rotation_of_minimal(seq, n)
    for i in 1:n-1
        rotated_seq = vcat(seq[i+1:end], seq[1:i])
        if rotated_seq < seq
            return false
        end
    end
    return true
end

function has_repeating_pattern(seq, n)
    for len in 1:div(n, 2)
        if n % len == 0
            pattern = seq[1:len]
            is_repeating = true
            for i in 1:div(n, len)
                if seq[(i-1)*len+1:i*len] != pattern
                    is_repeating = false
                    break
                end
            end
            if is_repeating
                return true
            end
        end
    end
    return false
end

function generate_minimal_non_repeating_sequences(d::Int, n::Int)
    minimal_sequences = Vector{Vector{Int}}()  
    seq = fill(0, n)  
    
    while seq != nothing
        if is_rotation_of_minimal(seq, n) && !has_repeating_pattern(seq, n)
            push!(minimal_sequences, copy(seq))  # Ajouter une copie de la séquence à la liste
        end
        seq = next_sequence(seq, d)
    end
    
    return minimal_sequences
end

function next_sequence(seq::Vector{Int}, d::Int)
    n = length(seq)
    for i in n:-1:1
        if seq[i] < d - 1
            seq[i] += 1
            for j in i+1:n
                seq[j] = 0
            end
            return seq
        end
    end
    return nothing
end

next_sequence (generic function with 1 method)

In [12]:
function dichotomy(g,z,xm,XM,epsi)
    t=(xm+XM)/2
    while XM-xm>epsi
        if g(t)>z
            XM=t
        else
            xm=t
        end
        t=(xm+XM)/2
    end
    return t
end

dichotomy (generic function with 1 method)

In [13]:
function suite_dadique(suite::Vector{Int}, d::Int,i)
    value = [sum(suite[(j+i-k)%i+1] * d^(j-1) for j in 1:i) for k in 1:i]
    return value
end

suite_dadique (generic function with 1 method)

In [14]:
function logdphiq(phi,orbx,i,N,n,epsi)
    lf=0
    qTx=phi_n_per(phi,orbx,i,0,n,0,epsi)
    for j in i:-1:1
        lf+=inf(phi.logds(interval(orbx[j]-epsi,orbx[j]+epsi),qTx))
        qTx=phi.image(interval(orbx[j]-epsi,orbx[j]+epsi),qTx)
    end
    return lf/i
end

logdphiq (generic function with 1 method)

In [15]:
function lowboundlambdaF(phi,T,n,N,M)
    d=T.degree
    lf=inf(phi.logds(0,phi_n(phi,T,interval(0,0),interval(0,0),n)))
    epsi=min(1/(2^N),1/(2*(1+T.maxd)*(T.maxd^M-1)))/10
    for i in 2:M
        g=x->compose(T.image,x,i)-x
        seq=generate_minimal_non_repeating_sequences(T.degree,i)
        for x in seq 
            image_orbx=suite_dadique(x,T.degree,i)
            orbx=[]
            xm=0
            xM=1
            for f in image_orbx
                a=dichotomy(g,f,xm,xM,epsi)
                push!(orbx,a)
                Ta=T.image(a)
                err=epsi*T.maxd
                xm=Ta-err
                xM=Ta+err
            end
            lf=max(lf,logdphiq(phi,T,orbx,i,N,n,epsi))
        end
    end
    return lf
end

lowboundlambdaF (generic function with 1 method)

## Bound on Lambda_u

In [61]:
function upperboundLu(T,N,m,Q,I)
    Indice=[i for i in 1:2^N]
    Nmaj=N
    bound=-Inf
    for z in 1:I
        len=length(Indice)
        l=[interval((i-1)/2^Nmaj,i/2^Nmaj) for i in Indice]
        r=[interval(0,0) for i in 1:len]
        infi=[Inf for i in 1:len]
        for i in 1:m
            r+=[log(T.d(x)) for x in l]
            infi=[min(sup(r[j])/i,infi[j]) for j in 1:len]
            l=[T.image(x) for x in l]
        end
        boundmaj=quantile(infi,1-1/2^Q)
        if bound>boundmaj 
            error("bound")
        end
        bound=boundmaj
        if z==I
            return maximum(infi)
        end
        Nmaj=Nmaj+Q
        Indicemaj=Indice[findall(x -> x > bound, infi)].*2^Q
        Indice=reduce(vcat,[Indicemaj.-i for i in 0:2^Q-1])
    end
end

upperboundLu (generic function with 2 methods)

In [60]:
function logdTseq(T,orbx,epsi)
    r=interval(0,0)
    i=length(orbx)
    for j in 1:i
        r+=log(T.d(interval(orbx[j]-epsi,orbx[j]+epsi)))
    end
    return inf(r)/i
end

logdTseq (generic function with 2 methods)

## Plot Holder regularity


In [62]:
function plotBoundHolderreg(phi,T,premierl,pas,nbl,n,N,m,M,nu,Nu,plt)
    "step 1: find approximation of the periodique orbit"
    d=T.degree
    epsi=1/(2^N)
    Orbite=Vector{Vector{Float64}}()
    push!(Orbite,[0])
    for i in 2:M
        g=x->compose(T.image,x,i)-x
        seq=generate_minimal_non_repeating_sequences(T.degree,i)
        for x in seq 
            image_orbx=suite_dadique(x,T.degree,i)
            orbx=[]
            xm=0
            xM=1
            for f in image_orbx
                a=dichotomy(g,f,xm,xM,epsi)
                push!(orbx,a)
                Ta=T.image(a)
                err=epsi*T.maxd
                xm=Ta-err
                xM=Ta+err
            end
        push!(Orbite,orbx) 
        end
    end
    "step 2: find Lu"
    lowLu=maximum([logdTseq(T,orbx,epsi) for orbx in Orbite])
    upLu=upperboundLu(T,10,14,1,20)
    print(lowLu,upLu)
    "step 3: find Lf"
    upLf=zeros(Float64,nbl)
    for p in 1:nbl
        l=premierl+(p-1)*pas
        upLf[p]=upboundlambdaF(phi(l),T,22,10,14,1,20)
    end
    lowLf=zeros(Float64,nbl)
    for p in 1:nbl
        l=premierl+(p-1)*pas
        lowLf[p]=maximum([logdphiq(phi(l),orbx,length(orbx),N,200,epsi) for orbx in Orbite])
    end
    lowReg=upLf./(-upLu)
    upReg=lowLf./(-lowLu)
    if plt==true
        L=[premierl + i*pas for i in 0:nbl-1]
        p=scatter(L,upReg,color=:blue)
        scatter!(p,L,lowReg,color=:red)
        times=Dates.format(now(),"yyyy-mm-dd_HH-MM-SS")
        ### savefig(homedir() * "/Bureau/plot/" * "graphe_$times.png")
        display(p)
    end
    return lowReg,upReg
end
    

plotBoundHolderreg (generic function with 1 method)

In [65]:
plotBoundHolderreg(phi1,T1(0.1),0.7,0.1,2,n,N,m,M,nu,Nu,true)

0.96634428221222920.966344299791131

LoadError: bound

## Examples of transformations

In [22]:
struct T
    image
    d
    maxd
    mind
    degree
end

In [23]:
T1(e)=T(x->2*x+e*sin(2*pi*x),x->2+2*pi*e*cos(2*pi*x),2+2*pi*e,2-2*pi*e,2)

T1 (generic function with 1 method)

## Examples of laws of reproduction

In [24]:
struct phi
    image
    logds
    exp
    maxexp
    maxdexp
    maxdx
end

In [25]:
phi1(l)=phi((x,s)->exp((s - 1) * exp(l + cos(2 * x * π))),(x,s)->log(exp(l + cos(2 * x * π)) * phi1(l).image(x, s)),x->exp(l+cos(2 * x * π)),exp(l+1),2*π*exp(l+1),2*π/exp(1))

phi1 (generic function with 1 method)

In [26]:
phi2(l)=phi((x,s)->exp((s - 1) * exp(l - cos(2 * x * π))),(x,s)->log(exp(l - cos(2 * x * π)) * phi2(l).image(x, s)),x->exp(l-cos(2 * x * π)),exp(l+1),2*π*exp(l+1),2*π/exp(1))

phi2 (generic function with 1 method)

## General settings

In [27]:
N = 14 # 2^N est le nb d'intervalles
n = 14 # le nb d'itérations de notre fonction majorante et minorante
m = 8
M = 8 # taille max des orbites périodiques

8

In [28]:
Nu=14
nu=8
Mu=8

8

## Main

In [29]:
e=0.1
"e doit etre inférieur à 1/(pi*2) soit environ 0.159"
l=0.6

l